In [ ]:
import pandas as pd

data =pd.read_csv(
    filepath_or_buffer='../data/test.csv',
    sep='\t',
    encoding='utf8',
    names=["ID", "Label", "Tweet"]
    )

data

In [ ]:
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import TweetTokenizer

tk = TweetTokenizer()
stop_words = set(stopwords.words('english'))

def token_pipeline(tweet):
    tweet_tokens = tk.tokenize(tweet)
    tweet_token_without_stopwords = list(filter(lambda token: token not in stop_words, tweet_tokens))
    return tweet_token_without_stopwords

data["Tweet_Token"] = data["Tweet"].apply(token_pipeline)

data

In [ ]:
import random

def find_synonyms(tweet):
    tweet_synonyms = []
    for token in tweet:
        word_synonyms=[]
        for word_synonym in wordnet.synsets(token):
            for word_lemma in word_synonym.lemmas():
                word_synonyms.append(word_lemma.name())
        if word_synonyms:
            tweet_synonyms.append(random.choice(list(set(word_synonyms))))
        else:
            tweet_synonyms.append(token)

    return tweet_synonyms

for count in range(10):
    data[f'synthetic {count}'] = data["Tweet_Token"].apply(find_synonyms)
    data[f'synthetic {count}'] = data[f'synthetic {count}'].apply(" ".join)